# Signal Spaces

## Utils
Utility functions used in the signal module.

- `ispower2::int->bool`: check if the input `int` is a power of 2
- `linespace::(signum a, int n)=>a->a->n->[a]`: function in the form of `f(a,b,n)` generates the sequence of equal spacing fractional numbers in interval `[a,b]` with length of `n`.
- `windowed :: int->Sequence[a]->Generator[Sequence[a],None,None]`: give rise to sliding windows on the input sequence of type `a`.

``` {literalinclude} ./Signal/Utils.py
---
name: Utils
language: python
linenos: True
caption: |
    `package Signal.Utils`
```


## Signal types

This package is about the types for the signal spaces. It defines `signum` as the (literal) data type. It is the type of the data in which the signal space package uses. It is more like a configuration parameter.

Currently, `signum` is set to `int|float|complex|str|None|tuple[()]`.

It also consists of the definitons of `Sig_0`, `Sig_1` and `Sig_3` representing 0-sig, 1-sig and 3-sig respectively.

- `sig_0`: list of type `signum`, so 0-sig in python is a list, but not an arbitrary list, it is a list constrainted by `signum`, a literal type in particular.
- `sig_1` list of `sig_0`. Each element is intended to be a block in the original signal. The usual use case is that blocks are non-overlapping blocks and the union is the original signal.
- `sig_3` is a list of triples of `sig_0`. The center `sig_0` in the triple is intended to be the corresponding block in `sig_1`, the left `sig_0` in the triple is the left extension of the center `sig_0` while the right one is the right extension.

``` {literalinclude} ./Signal/Types.py
---
name: Types
language: python
linenos: True
caption: |
    `package Signal.Types`
```

In [26]:
from Signal.Types import sig_0, sig_1, sig_3

a=sig_0([1,2,3,4])
b=sig_1([a,a,a])
c=sig_3([(a,a,a),(a,a,a),(a,a,a)])

In [27]:
def is_generic(t: type):
    return hasattr(t, "__origin__")

def is_generic_of(t: object, origin: object) -> bool:
    return is_generic(t) and t.__origin__ == origin  # type: ignore

In [28]:
is_generic_of(a,sig_0)

False

In [29]:
is_generic_of(sig_0,list)

True

OMG....., so `sig_0([1,2,3,4])` does nothing but `[1,2,3,4]`, ok, just for the type hint, i.e. static type checker, no complain....

In [30]:
print(a)
print(b)
print(c)

[1, 2, 3, 4]
[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]
[([1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]), ([1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]), ([1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4])]


## Padding
Padding package consists of `PaddingMethodNames` and `padding`:
- `PaddingMethodNames`: `Enum` type, defines the names of three padding methds, `ZERO`, `PERIODIC` and `SYMETRIC`.
- `padding`: a dictionary type, the key is padding method name, and the value is the function for padding corresponding to the method name, to be specific:
    - `padding[PaddingMethodNames.ZERO]=zero(n,v)`
    - `padding[PaddingMethodNames.PERIODIC]=periodic(n,v)`
    - `padding[PaddingMethodNames.SYMETRIC]=symetric(n,v)`
 
    where `n` is the length of the extension, `v` is the signal with type `Sig_0`. The return type is a pair of type `(Sig_0,Sig_0)`, correponding to the left and right extension repectively.

``` {literalinclude} ./Signal/Padding.py
---
name: Padding
language: python
linenos: True
caption: |
    `package Signal.Padding`
```

In [31]:
from Signal.Padding import PaddingMethodNames as pmn

for i in pmn: print(i)

PaddingMethod.ZERO
PaddingMethod.PERIODIC
PaddingMethod.SYMETRIC


In [32]:
from Signal.Padding import padding
from Signal.Types import sig_0

a=sig_0([x for x in range(1,9)])
foo = map((lambda x: padding[x](4,a)), pmn)
print(a)
for i,j in zip(pmn,foo): print(str(i)+":",j)

[1, 2, 3, 4, 5, 6, 7, 8]
PaddingMethod.ZERO: ([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0])
PaddingMethod.PERIODIC: ([5, 6, 7, 8], [1, 2, 3, 4])
PaddingMethod.SYMETRIC: ([4, 3, 2, 1], [8, 7, 6, 5])


## Signal contexts
`Sig_0`, `Sig_1` and `Sig_3` are the container. We need to attach contexts to them. There are two classes of contexts:
computational contexts and iteration contexts. 

The computational context consists of:
- functor: it is just a map, so it models a single for-loop.
- applicative: multiple maps, so it models multiple independent for-loops.
- monad: main method is the the bind function. It also a functor and an applicative. A chain of bind functions can emulate multiple layer for-loops.

The computation results are retained within the container.

The iteration context has two classes:
- foldable
- traversable

All contexts from the `Signal` package are derived from the list contexts defined in the package `Contexts`, the types are restricted to `sig_0`, `sig_1` and `sig_3`:
- `Sigor`: stands for signal functor.
- `Sigive`: Signal applicative functor.
- `SigMonad`: Signal monad, it is also a `Sigor` and `Sigive`.

In [33]:
from Signal.Contexts import Sigor

s=Sigor(a)
s.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0']

eh...., problem with static checker or problem with our design?

In [34]:
print(id(a),id(s))

140559586094080 140559585865600


So, `Sigor` made a copy of `a`.....as expected. Each `fmap` creates two new copies (one from `map`, the other from the constructor of `Sigor`). This seems to be a draw back of `Sigor`. But using `map` directly seems to have the same effect. Something to think about....

In [35]:
s.fmap(float).fmap(str).fmap(print)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0


[None, None, None, None, None, None, None, None]

In [36]:
b=[a,a,a]
s=Sigor(b)
sum(s.fmap(sum).fmap(float).fmap(abs))

108.0

In [37]:
from Signal.Contexts import Sigive

s=Sigive(a)

In [38]:
s.pure(3)

[3]

In [39]:
s.ap([abs,float]).fmap(lambda x: -x)

[-1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 -1.0,
 -2.0,
 -3.0,
 -4.0,
 -5.0,
 -6.0,
 -7.0,
 -8.0]

In [40]:
from Signal.Contexts import SigLiftA2

a=[1,2]
b=[x for x in range(9,17)]
list(SigLiftA2((lambda x, y: x+y),a,b))

[10, 11, 12, 13, 14, 15, 16, 17, 11, 12, 13, 14, 15, 16, 17, 18]

In [41]:
from Signal.Contexts import SigMonad

a=[x for x in range(1,9)]
m=SigMonad(a)

In [42]:
m.then([1,2])

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

In [43]:
m.fmap(float)


[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [44]:
m.ap([float,(lambda x: x+2)])

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 3, 4, 5, 6, 7, 8, 9, 10]

In [45]:
m.bind(lambda x:[x-1,x,x+1])

[0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9]

In [46]:
m.bind(lambda x:[[x-1,x,x+1]])

[[0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9]]

## The model
The model package provides two morphisoms `blocks` and `oblocks`: 
- `blocks` - type definition: `blocks::int->sig_0->sig_1`, function form - `blocks(n,v)`. This maps an input signal `v` in type `sig_0` to `sig_1`, each with length of `n`.
- `oblocks` - type definition: `oblocks::PaddingMethd->int->sig_1->sig_3`, function  form `f(pm,n,b)`. `oblocks` maps `b` in type `sig_1` to `sig_3` with `PaddingMethod` pm and extension length `n`.

In [47]:
from Signal.Morphisms import blocks

a=sig_0([x for x in range(1,9)])
blocks(2,a)

[[1, 2], [3, 4], [5, 6], [7, 8]]

In [48]:
from Signal.Morphisms import oblocks, getExtensions, leftEdge, rightEdge
from Signal.Padding import PaddingMethodNames
from Contexts.Foldable import scanl, scanr

from functools import partial

b=blocks(4,a)
oblocks(PaddingMethodNames.ZERO,2,b)

[([0, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [0, 0])]

In [49]:
oblocks(PaddingMethodNames.PERIODIC,2,b)

[([7, 8], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [1, 2])]

In [50]:
oblocks(PaddingMethodNames.SYMETRIC,2,b)

[([2, 1], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [8, 7])]

## Algorithms
Algorithms are endomorphisms in `sig_0` or `(sig_0,sig_0,sig_0)`. For the case of `sig_0`, each algorithm maps a `sig_0` concrete object to a another `sig_0` concrete object (here the name object is in the sense of programming, e.g. `1` is an object of `int`). The same is true for the case of `(sig_0,sig_0,sig_0)`.

## Schemes
Schemes are the mappings between `sig_1` and `sig_3`. So they are the mappings in the category level.
- `sig_1` to `sig_1` or `sig_3` to `sig_3`
- `sig_1` to `sig_3` or `sig_3` to `sig_1`
- Tree structure for the adaptive schemes (to be determined).